In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# List physical devices
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))



Num GPUs Available:  0


In [3]:
# Path dataset
data_dir = r'D:\Kuliah\Semester 7\Computer Vision\Architecture\DatasetHFER'
input_shape = (224, 224, 3)
num_classes = 8  # Jumlah kelas

In [4]:
# Fungsi untuk memuat dan melakukan CLAHE pada gambar
def load_and_preprocess_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (224, 224))
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(gray_img)
    processed_img = cv2.cvtColor(cl, cv2.COLOR_GRAY2RGB) / 255.0
    return processed_img

In [6]:
# Fungsi untuk memuat dataset dari folder
def load_dataset(data_dir):
    images = []
    labels = []
    class_names = sorted(os.listdir(data_dir))
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.endswith('.png'):
                file_path = os.path.join(class_dir, file_name)
                img = load_and_preprocess_image(file_path)
                images.append(img)
                labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

In [7]:
# Memuat data dan membagi menjadi training dan testing
images, labels, class_names = load_dataset(data_dir)
labels = to_categorical(labels, num_classes=num_classes)  # One-hot encoding
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
# Augmentasi
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

In [10]:
# Memuat model VGG19 tanpa fully connected layer di akhir
base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers:
    layer.trainable = False

In [11]:
# Menambahkan layer kustom
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [12]:
# Mengompilasi model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Melatih model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=(X_test, y_test)
)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 449s 3s/step - accuracy: 0.1878 - loss: 2.1594 - val_accuracy: 0.2455 - val_loss: 1.9636


In [14]:
# Evaluasi model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

35/35 ━━━━━━━━━━━━━━━━━━━━ 99s 3s/step


In [15]:
# Menampilkan laporan klasifikasi
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
classification_rep = classification_report(y_true, y_pred, target_names=class_names)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

In [16]:
print(f"Akurasi: {accuracy}")
print(f"Presisi: {precision}")
print(f"Recall: {recall}")
print("Laporan Klasifikasi:")
print(classification_rep)

Akurasi: 0.24550359712230216
Presisi: 0.06038061442737702
Recall: 0.24550359712230216
Laporan Klasifikasi:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       199
    contempt       0.00      0.00      0.00        42
     disgust       0.00      0.00      0.00        96
        fear       0.00      0.00      0.00       107
   happiness       0.25      1.00      0.39       273
  neutrality       0.00      0.00      0.00        95
     sadness       0.00      0.00      0.00       149
    surprise       0.00      0.00      0.00       151

    accuracy                           0.25      1112
   macro avg       0.03      0.12      0.05      1112
weighted avg       0.06      0.25      0.10      1112

